In [220]:
import sys
!{sys.executable} -m pip install lightfm
import random

import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

In [221]:
p = 0.50

In [222]:
df_playlist = pd.read_csv('./data/spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and random.random() > p)

In [223]:
df_playlist.shape

(6442840, 4)

In [224]:
df_playlist.columns = df_playlist.columns.str.replace('"', '')
df_playlist.columns = df_playlist.columns.str.replace('name', '')
df_playlist.columns = df_playlist.columns.str.replace(' ', '')
df_playlist.columns
df_playlist.head()

,user_id,artist,track,playlist
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,Band On The Run,HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Bright Side,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Crowded House,Four Seasons In One Day,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,The Breakers,Get Lost Get High Get Sick,HARD ROCK 2010


In [225]:
# Adding test user and seeing what the output will be.
temp_user_id = '00000000000000000000000000000000'
temp_user_artists = ['Eminem', 'Kanye West', '50 Cent', 'Drake', 'Kendrick Lamar', 'Future', 'Lil Baby', 'Snoop Dogg', 'Post Malone', 'Lil Wayne', 'Kid Cudi']
temp_track_name = 'testtrackname'
temp_playlist_name = 'testplaylist'

new_rows = []
for i in range(20000):
    temp_artist = random.choice(temp_user_artists)
    new_row = {
        'user_id': temp_user_id,
        'artist': temp_artist,
        'track': temp_track_name,
        'playlist': temp_playlist_name
    }
    new_rows.append(new_row)

temp_df = pd.DataFrame(new_rows) 
df_playlist = pd.concat([df_playlist, temp_df], ignore_index = True)
df_last = df_playlist.tail(10)
df_last

df_playlist.loc[df_playlist['user_id'] == temp_user_id]

,user_id,artist,track,playlist
6442840,00000000000000000000000000000000,Kendrick Lamar,testtrackname,testplaylist
6442841,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442842,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442843,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442844,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
...,...,...,...,...
6462835,00000000000000000000000000000000,Kendrick Lamar,testtrackname,testplaylist
6462836,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
6462837,00000000000000000000000000000000,Snoop Dogg,testtrackname,testplaylist
6462838,00000000000000000000000000000000,Kid Cudi,testtrackname,testplaylist


In [226]:
# Keeping artists that appear with a minimum frequency. 
# Only keeping artists with high frequency than 50

In [227]:
df_playlist = df_playlist.groupby('artist').filter(lambda x : len(x) >= 50)
# df_playlist
df_playlist.loc[df_playlist['user_id'] == temp_user_id]

,user_id,artist,track,playlist
6442840,00000000000000000000000000000000,Kendrick Lamar,testtrackname,testplaylist
6442841,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442842,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442843,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442844,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
...,...,...,...,...
6462835,00000000000000000000000000000000,Kendrick Lamar,testtrackname,testplaylist
6462836,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
6462837,00000000000000000000000000000000,Snoop Dogg,testtrackname,testplaylist
6462838,00000000000000000000000000000000,Kid Cudi,testtrackname,testplaylist


In [228]:
# Keeping users with at least 10 unique artists in their playlists to lessen
# the impact of COLD START PROBLEM

In [229]:
df_playlist = df_playlist[df_playlist.groupby('user_id').artist.transform('nunique') >= 10]
df_playlist.loc[df_playlist['user_id'] == temp_user_id]

,user_id,artist,track,playlist
6442840,00000000000000000000000000000000,Kendrick Lamar,testtrackname,testplaylist
6442841,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442842,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442843,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
6442844,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
...,...,...,...,...
6462835,00000000000000000000000000000000,Kendrick Lamar,testtrackname,testplaylist
6462836,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
6462837,00000000000000000000000000000000,Snoop Dogg,testtrackname,testplaylist
6462838,00000000000000000000000000000000,Kid Cudi,testtrackname,testplaylist


In [230]:
# Grouping by the frequency count for each users artists (# of times that an
# artist appeared in playlists created by a user)

In [231]:
size = lambda x : len(x)
df_freq = df_playlist.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

df_freq.loc[df_freq['user_id'] == temp_user_id]

,user_id,artist,freq
9,00000000000000000000000000000000,Post Malone,1957
4,00000000000000000000000000000000,Kanye West,1863
5,00000000000000000000000000000000,Kendrick Lamar,1860
0,00000000000000000000000000000000,50 Cent,1841
8,00000000000000000000000000000000,Lil Wayne,1824
6,00000000000000000000000000000000,Kid Cudi,1785
10,00000000000000000000000000000000,Snoop Dogg,1784
3,00000000000000000000000000000000,Future,1783
1,00000000000000000000000000000000,Drake,1770
2,00000000000000000000000000000000,Eminem,1769


In [232]:
# Creating a data frame for artists and artist id
# Artist to artist id mapping

In [233]:
df_artist = pd.DataFrame(df_freq['artist'].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
df_artist.head()

,artist_id,artist
0,0,Post Malone
1,1,Kanye West
2,2,Kendrick Lamar
3,3,50 Cent
4,4,Lil Wayne


In [234]:
df_artist.shape

(14305, 2)

In [235]:
# Adding artist_id to the frequence data frame
df_freq = pd.merge(df_freq, df_artist, how='inner', on='artist')

In [236]:
#########
# Using the lightfm library to run a traditional MF (matrix factorization) model
# since the dataset doesn't include any user or artist features
# the library allows builing of a hybrid model as well. (CF AND CBF?)
# examples: https://github.com/lyst/lightfm/blob/master/examples/

In [237]:
# Don't know what the
# Creating an interaction matrix df from transactional type interactions
def create_interaction_matrix(df, user_col, item_col, rating_col, norm=False, threshold=None):
    interactions = df.groupby([user_col, item_col])[rating_col].sum().unstack().reset_index().fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [238]:
# Function to create a user dictionary based on their index and number in
# interaction dataset
def create_user_dict(interactions):
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [239]:
# Function to create an item dictionary based on their item_id and item name
def create_item_dict(df, id_col, name_col):
    item_dict = {}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i, name_col]
    return item_dict

In [240]:
# Runs the matrix factorization algorithm and returns the trained model
def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30, n_jobs = 4):
    model = LightFM(no_components = n_components, loss=loss, k=k)
    model.fit(x, epochs=epoch, num_threads = n_jobs)
    return model

In [241]:
# Produces user recommendations, prints the list of items the user already 
# prefers, prints the list of N recommended items which user will hopefully 
# prefer.
def sample_recommendation_user(model, interactions, user_id, user_dict, item_dict, threshold = 0, nrec_items = 10, show = True):
    n_users, n_items = interactions.shape # indices
    user_x = user_dict[user_id] # getting the specific user
    scores = pd.Series(model.predict(user_x, np.arange(n_items))) # model prediction
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:]
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [242]:
# Prepping the model inputs

In [243]:
# Create interaction matrix
interactions = create_interaction_matrix(df=df_freq, user_col="user_id", item_col="artist_id", rating_col='freq', norm=False, threshold=None)
interactions.head()

artist_id,0,1,2,3,4,5,6,7,8,9,...,14295,14296,14297,14298,14299,14300,14301,14302,14303,14304
user_id,,,,,,,,,,,,,,,,,,,,,
00000000000000000000000000000000,1957.0,1863.0,1860.0,1841.0,1824.0,1785.0,1784.0,1783.0,1770.0,1769.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00055176fea33f6e027cd3302289378b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0007f3dd09c91198371454c608d47f22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000b0f32b5739f052b9d40fcc5c41079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c11a16c89aa4b14b328080f5954ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [244]:
interactions.shape

(12931, 14305)

In [245]:
# temp

In [246]:
# Create user dict
user_dict = create_user_dict(interactions=interactions)

In [247]:
# Create item dict
artists_dict = create_item_dict(df=df_artist, id_col="artist_id", name_col="artist")

In [248]:
# Train-test split
x = sparse.csr_matrix(interactions.values)
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=None)

In [249]:
# How does a matrix factorization model work?
# https://developers.google.com/machine-learning/recommendation/collaborative/matrix

In [250]:
# Train the matrix factorization model
# Can do hyper-parameter tuning for better results
%time
model = runMF(interactions = train, n_components=30, loss='warp', k=15, epoch=30, n_jobs=4)

Wall time: 0 ns


In [251]:
# Evaluation metrics
# Compute AUC score for train set
train_auc = auc_score(model, train, num_threads=4).mean()
print('Train AUC: %s' % train_auc)

Train AUC: 0.96671736


In [252]:
# Compute AUC score test set
# the parameter train_interactions allows you to exlude known positives in
# training set from the predictions and score calculations
# this is to avoid re-recommending the items the user has already interacted with
test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
print('Test AUC: %s' % test_auc)

Test AUC: 0.96734715


In [253]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

In [254]:
print('train Precision %.2f, test Precision %.2f.' % (train_precision, test_precision))

train Precision 0.40, test Precision 0.18.


In [255]:
random_cell = interactions.sample()
random_user_id = random_cell.index[0]

rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = temp_user_id, 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Lil Baby
2- Eminem
3- Drake
4- Future
5- Snoop Dogg
6- Kid Cudi
7- Lil Wayne
8- 50 Cent
9- Kendrick Lamar
10- Kanye West
11- Post Malone

 Recommended Items:
1- Big Sean
2- Kid Ink
3- YG
4- J. Cole
5- T.I.
6- JAY Z
7- Chris Brown
8- A$AP Rocky
9- Mike Will Made It
10- Nicki Minaj
11- Ariana Grande
12- Schoolboy Q
13- The Weeknd
14- Ty Dolla $ign
15- 2 Chainz
